In [0]:
%matplotlib inline


Compile ONNX Models
===================
**Author**: `Joshua Z. Zhang <https://zhreshold.github.io/>`_

This article is an introductory tutorial to deploy ONNX models with Relay.

For us to begin with, ONNX package must be installed.

A quick solution is to install protobuf compiler, and

.. code-block:: bash

    pip install onnx --user

or please refer to offical site.
https://github.com/onnx/onnx



In [6]:
!pip install onnx

In [0]:
#from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)

In [7]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    ! gsutil cp "gs://tvm-fcrc-binariesd5fce43e-8373-11e9-bfb6-0242ac1c0002/tvm.tar.gz" /tmp/tvm.tar.gz
    ! mkdir -p /tvm
    ! tar -xf /tmp/tvm.tar.gz --strip-components=4 --directory /tvm
    ! ls -la /tvm
    ! bash /tvm/package.sh
    ! pip install onnx 
    ! sudo apt install clang-format
    # ! pip install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
    # ! pip3 install https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl
    # Add TVM to the Python path.
    import sys
    import os
    sys.path.append('/tvm/python')
    sys.path.append('/tvm/topi/python')
    sys.path.append('/tvm/vta/python')
    os.environ['TVM_HOME'] = '/tvm'
else:
    print("Notebook executing locally, skipping Colab setup ...")

Copying gs://tvm-fcrc-binariesd5fce43e-8373-11e9-bfb6-0242ac1c0002/tvm.tar.gz...
| [1 files][119.5 MiB/119.5 MiB]                                                
Operation completed over 1 objects/119.5 MiB.                                    
total 164
drwxr-xr-x 21 root root  4096 Apr 15 06:42 .
drwxr-xr-x  1 root root  4096 Apr 15 06:41 ..
drwx------  8 root root  4096 May 31  2019 3rdparty
drwx------ 12 root root  4096 May 31  2019 apps
drwx------  3 root root  4096 Jun 19  2019 build
drwx------  4 root root  4096 May 31  2019 cmake
-rw-------  1 root root 11053 Jun 19  2019 CMakeLists.txt
drwx------  6 root root  4096 May 31  2019 conda
-rw-------  1 root root  5736 Jun 19  2019 CONTRIBUTORS.md
drwx------  3 root root  4096 May 31  2019 docker
drwx------ 11 root root  4096 May 31  2019 docs
drwx------  4 root root  4096 May 31  2019 golang
drwx------  3 root root  4096 May 31  2019 include
-rw-------  1 root root 10607 Jun 19  2019 Jenkinsfile
drwx------  6 root root  4096 May 31 

In [8]:
if IN_COLAB:
    ! cd /; git clone https://github.com/uwsampl/relay-aot
    sys.path.append('/relay-aot')

Cloning into 'relay-aot'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 464 (delta 37), reused 33 (delta 18), pack-reused 397
Receiving objects: 100% (464/464), 92.71 KiB | 513.00 KiB/s, done.
Resolving deltas: 100% (261/261), done.


In [0]:
import onnx
import numpy as np
import tvm
import tvm.relay as relay
from tvm.contrib.download import download_testdata
import tvm.relay.testing
from tvm.relay.expr_functor import ExprMutator, ExprVisitor
import logging
logging.disable(logging.WARNING)

Load pretrained ONNX model
---------------------------------------------
The example super resolution model used here is exactly the same model in onnx tutorial
http://pytorch.org/tutorials/advanced/super_resolution_with_caffe2.html
we skip the pytorch model construction part, and download the saved onnx model



In [12]:
%pwd

'/content'

In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [18]:
%pwd

'/content'

In [0]:
#model_url = ''.join(['https://gist.github.com/zhreshold/',
#                     'bcda4716699ac97ea44f791c24310193/raw/',
#                     '93672b029103648953c4e5ad3ac3aadf346a4cdc/',
#                     'super_resolution_0.2.onnx'])
#model_path = download_testdata(model_url, 'super_resolution.onnx', module='onnx')
# now you have super_resolution.onnx on disk
model_path='gdrive/My Drive/mnist.onnx'
onnx_mnist = onnx.load(model_path)

In [34]:
onnx_mnist

ir_version: 3
producer_name: "CNTK"
producer_version: "2.5.1"
domain: "ai.cntk"
model_version: 1
graph {
  node {
    input: "Parameter193"
    input: "Parameter193_reshape1_shape"
    output: "Parameter193_reshape1"
    name: "Times212_reshape1"
    op_type: "Reshape"
    doc_string: ""
    domain: ""
  }
  node {
    input: "Input3"
    input: "Parameter5"
    output: "Convolution28_Output_0"
    name: "Convolution28"
    op_type: "Conv"
    attribute {
      name: "kernel_shape"
      ints: 5
      ints: 5
      type: INTS
    }
    attribute {
      name: "strides"
      ints: 1
      ints: 1
      type: INTS
    }
    attribute {
      name: "auto_pad"
      s: "SAME_UPPER"
      type: STRING
    }
    attribute {
      name: "group"
      i: 1
      type: INT
    }
    attribute {
      name: "dilations"
      ints: 1
      ints: 1
      type: INTS
    }
    doc_string: ""
    domain: ""
  }
  node {
    input: "Convolution28_Output_0"
    input: "Parameter6"
    output: "Plus30_

In [67]:
#onnx_mnist = onnx.load('mnist.onnx')
func, params = relay.frontend.from_onnx(onnx_mnist, shape={ 'Input3': (1, 1, 28, 28) })
print(func)

v0.0.1
def @main(%Input3: Tensor[(1, 1, 28, 28), float32], %Parameter5: Tensor[(8, 1, 5, 5), float32], %Parameter6: Tensor[(8, 1, 1), float32], %Parameter87: Tensor[(16, 8, 5, 5), float32], %Parameter88: Tensor[(16, 1, 1), float32], %Parameter193: Tensor[(16, 4, 4, 10), float32], %Parameter194: Tensor[(1, 10), float32]) -> Tensor[(1, 10), float32] {
  %0 = nn.conv2d(%Input3, %Parameter5, kernel_size=[5, 5]) /* ty=Tensor[(1, 8, 24, 24), float32] */
  %1 = add(%0, %Parameter6) /* ty=Tensor[(1, 8, 24, 24), float32] */
  %2 = nn.relu(%1) /* ty=Tensor[(1, 8, 24, 24), float32] */
  %3 = nn.max_pool2d(%2, pool_size=[2, 2], strides=[2, 2]) /* ty=Tensor[(1, 8, 12, 12), float32] */
  %4 = nn.conv2d(%3, %Parameter87, kernel_size=[5, 5]) /* ty=Tensor[(1, 16, 8, 8), float32] */
  %5 = add(%4, %Parameter88) /* ty=Tensor[(1, 16, 8, 8), float32] */
  %6 = nn.relu(%5) /* ty=Tensor[(1, 16, 8, 8), float32] */
  %7 = nn.max_pool2d(%6, pool_size=[3, 3], strides=[3, 3]) /* ty=Tensor[(1, 16, 2, 2), float32] 

In [0]:
# print(params)

In [69]:
i = 0 
def ann(*args):
    global i
    i += 1
    return f" <expression: {i}>"

print(func.astext(show_meta_data=True, annotate=ann))

v0.0.1
def @main(%Input3: Tensor[(1, 1, 28, 28), float32], %Parameter5: Tensor[(8, 1, 5, 5), float32], %Parameter6: Tensor[(8, 1, 1), float32], %Parameter87: Tensor[(16, 8, 5, 5), float32], %Parameter88: Tensor[(16, 1, 1), float32], %Parameter193: Tensor[(16, 4, 4, 10), float32], %Parameter194: Tensor[(1, 10), float32]) -> Tensor[(1, 10), float32] {
  %0 = nn.conv2d <expression: 1>(%Input3, %Parameter5, kernel_size=[5, 5]) <expression: 2>
  %1 = add <expression: 3>(%0, %Parameter6) <expression: 4>
  %2 = nn.relu <expression: 5>(%1) <expression: 6>
  %3 = nn.max_pool2d <expression: 7>(%2, pool_size=[2, 2], strides=[2, 2]) <expression: 8>
  %4 = nn.conv2d <expression: 1>(%3, %Parameter87, kernel_size=[5, 5]) <expression: 9>
  %5 = add <expression: 3>(%4, %Parameter88) <expression: 10>
  %6 = nn.relu <expression: 5>(%5) <expression: 11>
  %7 = nn.max_pool2d <expression: 7>(%6, pool_size=[3, 3], strides=[3, 3]) <expression: 12>
  %8 = reshape <expression: 13>(%7, newshape=[1, 256]) <expres

In [0]:
seq = relay.transform.Sequential([
    relay.transform.InferType(),
    relay.transform.SimplifyInference(),
    relay.transform.FoldConstant(),
    relay.transform.EliminateCommonSubexpr(),
    relay.transform.AlterOpLayout()
])

In [73]:
#onnx_mnist = onnx.load('mnist.onnx')
func, params = relay.frontend.from_onnx(onnx_mnist, shape={ 'Input3': (1, 1, 28, 28) })
print(func)

v0.0.1
def @main(%Input3: Tensor[(1, 1, 28, 28), float32], %Parameter5: Tensor[(8, 1, 5, 5), float32], %Parameter6: Tensor[(8, 1, 1), float32], %Parameter87: Tensor[(16, 8, 5, 5), float32], %Parameter88: Tensor[(16, 1, 1), float32], %Parameter193: Tensor[(16, 4, 4, 10), float32], %Parameter194: Tensor[(1, 10), float32]) -> Tensor[(1, 10), float32] {
  %0 = nn.conv2d(%Input3, %Parameter5, kernel_size=[5, 5]) /* ty=Tensor[(1, 8, 24, 24), float32] */
  %1 = add(%0, %Parameter6) /* ty=Tensor[(1, 8, 24, 24), float32] */
  %2 = nn.relu(%1) /* ty=Tensor[(1, 8, 24, 24), float32] */
  %3 = nn.max_pool2d(%2, pool_size=[2, 2], strides=[2, 2]) /* ty=Tensor[(1, 8, 12, 12), float32] */
  %4 = nn.conv2d(%3, %Parameter87, kernel_size=[5, 5]) /* ty=Tensor[(1, 16, 8, 8), float32] */
  %5 = add(%4, %Parameter88) /* ty=Tensor[(1, 16, 8, 8), float32] */
  %6 = nn.relu(%5) /* ty=Tensor[(1, 16, 8, 8), float32] */
  %7 = nn.max_pool2d(%6, pool_size=[3, 3], strides=[3, 3]) /* ty=Tensor[(1, 16, 2, 2), float32] 

In [74]:
mod = func # relay.Module({"main": func})
with relay.build_config(opt_level=3):
    with tvm.target.create("llvm"):
        mod = seq(mod)
        print(mod["main"])

v0.0.1
fn (%Input3: Tensor[(1, 1, 28, 28), float32], %Parameter5: Tensor[(8, 1, 5, 5), float32], %Parameter6: Tensor[(8, 1, 1), float32], %Parameter87: Tensor[(16, 8, 5, 5), float32], %Parameter88: Tensor[(16, 1, 1), float32], %Parameter193: Tensor[(16, 4, 4, 10), float32], %Parameter194: Tensor[(1, 10), float32]) -> Tensor[(1, 10), float32] {
  %0 = layout_transform(%Input3, src_layout="NCHW", dst_layout="NCHW1c") /* ty=Tensor[(1, 1, 28, 28, 1), float32] */
  %1 = layout_transform(%Parameter5, src_layout="OIHW", dst_layout="OIHW1i8o") /* ty=Tensor[(1, 1, 5, 5, 1, 8), float32] */
  %2 = nn.contrib_conv2d_NCHWc(%0, %1, meta[relay.attrs.Conv2DAttrs][0]) /* ty=Tensor[(1, 1, 24, 24, 8), float32] */
  %3 = layout_transform(%Parameter6, src_layout="CHW", dst_layout="CHW8c") /* ty=Tensor[(1, 1, 1, 8), float32] */
  %4 = add(%2, %3) /* ty=Tensor[(1, 1, 24, 24, 8), float32] */
  %5 = nn.relu(%4) /* ty=Tensor[(1, 1, 24, 24, 8), float32] */
  %6 = nn.max_pool2d(%5, pool_size=[2, 2], strides=[2, 2

In [115]:
#onnx_mnist = onnx.load('mnist.onnx')
#func, params = relay.testing.mlp.get_workload(1) #relay.frontend.from_onnx(onnx_mnist, shape={ 'Input3': (1, 1, 28, 28) })
func, params = relay.frontend.from_onnx(onnx_mnist, shape={ 'Input3': (1, 1, 28, 28) })
print(func['main'])

v0.0.1
fn (%Input3: Tensor[(1, 1, 28, 28), float32], %Parameter5: Tensor[(8, 1, 5, 5), float32], %Parameter6: Tensor[(8, 1, 1), float32], %Parameter87: Tensor[(16, 8, 5, 5), float32], %Parameter88: Tensor[(16, 1, 1), float32], %Parameter193: Tensor[(16, 4, 4, 10), float32], %Parameter194: Tensor[(1, 10), float32]) -> Tensor[(1, 10), float32] {
  %0 = nn.conv2d(%Input3, %Parameter5, kernel_size=[5, 5]) /* ty=Tensor[(1, 8, 24, 24), float32] */
  %1 = add(%0, %Parameter6) /* ty=Tensor[(1, 8, 24, 24), float32] */
  %2 = nn.relu(%1) /* ty=Tensor[(1, 8, 24, 24), float32] */
  %3 = nn.max_pool2d(%2, pool_size=[2, 2], strides=[2, 2]) /* ty=Tensor[(1, 8, 12, 12), float32] */
  %4 = nn.conv2d(%3, %Parameter87, kernel_size=[5, 5]) /* ty=Tensor[(1, 16, 8, 8), float32] */
  %5 = add(%4, %Parameter88) /* ty=Tensor[(1, 16, 8, 8), float32] */
  %6 = nn.relu(%5) /* ty=Tensor[(1, 16, 8, 8), float32] */
  %7 = nn.max_pool2d(%6, pool_size=[3, 3], strides=[3, 3]) /* ty=Tensor[(1, 16, 2, 2), float32] */
  %

In [0]:
opt_level = 3
target = "llvm"
with relay.build_config(opt_level):
  graph, lib, params = relay.build_module.build(func['main'], target, params=params)
  #intrp = relay.build_module.create_executor('graph', func, tvm.cpu(0), target)

In [126]:
func

ModuleNode( {GlobalVar(main): FunctionNode([Var(Input3, ty=TensorType([1, 1, 28, 28], float32)), Var(Parameter5, ty=TensorType([8, 1, 5, 5], float32)), Var(Parameter6, ty=TensorType([8, 1, 1], float32)), Var(Parameter87, ty=TensorType([16, 8, 5, 5], float32)), Var(Parameter88, ty=TensorType([16, 1, 1], float32)), Var(Parameter193, ty=TensorType([16, 4, 4, 10], float32)), Var(Parameter194, ty=TensorType([1, 10], float32))], TensorType([1, 10], float32), CallNode(Op(add), [CallNode(Op(nn.dense), [CallNode(Op(reshape), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.relu), [CallNode(Op(add), [CallNode(Op(nn.conv2d), [CallNode(Op(nn.max_pool2d), [CallNode(Op(nn.relu), [CallNode(Op(add), [CallNode(Op(nn.conv2d), [Var(Input3, ty=TensorType([1, 1, 28, 28], float32)), Var(Parameter5, ty=TensorType([8, 1, 5, 5], float32))], relay.attrs.Conv2DAttrs(0x4a31740), [TensorType([1, 1, 28, 28], float32), TensorType([8, 1, 5, 5], float32)]), Var(Parameter6, ty=TensorType([8, 1, 1], float32))], (nullptr), [

In [127]:
graph

'{\n  "nodes": [\n    {\n      "op": "null", \n      "name": "Input3", \n      "inputs": []\n    }, \n    {\n      "op": "null", \n      "name": "Parameter5", \n      "inputs": []\n    }, \n    {\n      "op": "null", \n      "name": "Parameter6", \n      "inputs": []\n    }, \n    {\n      "op": "null", \n      "name": "Parameter87", \n      "inputs": []\n    }, \n    {\n      "op": "null", \n      "name": "Parameter88", \n      "inputs": []\n    }, \n    {\n      "op": "null", \n      "name": "Parameter193", \n      "inputs": []\n    }, \n    {\n      "op": "null", \n      "name": "Parameter194", \n      "inputs": []\n    }, \n    {\n      "op": "tvm_op", \n      "name": "fused_layout_transform_4", \n      "attrs": {\n        "num_outputs": "1", \n        "num_inputs": "1", \n        "func_name": "fused_layout_transform_4", \n        "flatten_data": "0"\n      }, \n      "inputs": [\n        [\n          0, \n          0, \n          0\n        ]\n      ]\n    }, \n    {\n      "op": 

In [128]:
lib.get_source()

'; ModuleID = \'fused_reshape_transpose\'\nsource_filename = "fused_reshape_transpose"\ntarget datalayout = "e-m:e-i64:64-f80:128-n8:16:32:64-S128"\ntarget triple = "x86_64-pc-linux-gnu"\n\n%0 = type { i32*, i32 }\n%1 = type { i8*, %2, i32, %3, i64*, i64*, i64 }\n%2 = type { i32, i32 }\n%3 = type { i8, i8, i16 }\n%4 = type { i8*, i8* }\n%5 = type { i8*, i8* }\n%6 = type { i8*, i8* }\n%7 = type { i8*, i8*, i8*, i8* }\n%8 = type { i8*, i8*, float* }\n%9 = type { i8*, i8* }\n%10 = type { i8*, i8* }\n%11 = type { i8*, i8*, i8*, i8* }\n%12 = type { i8*, i8* }\n%13 = type { i8*, i8* }\n\n@__TVMAPISetLastError = linkonce dllexport local_unnamed_addr global void (i8*)* null, align 8\n@__TVMBackendParallelLaunch = linkonce dllexport local_unnamed_addr global i32 (i32 (i32, %0*, i8*)*, i8*, i32)* null, align 8\n@.str = private constant [76 x i8] c"Assert fail: (num_args == 2), fused_reshape_transpose: num_args should be 2\\00", align 1\n@.str.1 = private constant [196 x i8] c"Assert fail: ((((1 

In [54]:
#mod = func # relay.Module({"main": func})
opt_level=3
target = "llvm"
with relay.build_config(opt_level):
    graph, lib, params = relay.build(func, target, params=params)

TVMError: ignored

Load a test image
---------------------------------------------
A single cat dominates the examples!



In [0]:
from PIL import Image
img_url = 'https://github.com/dmlc/mxnet.js/blob/master/data/cat.png?raw=true'
img_path = download_testdata(img_url, 'cat.png', module='data')
img = Image.open(img_path).resize((224, 224))
img_ycbcr = img.convert("YCbCr")  # convert to YCbCr
img_y, img_cb, img_cr = img_ycbcr.split()
x = np.array(img_y)[np.newaxis, np.newaxis, :, :]

Compile the model with relay
---------------------------------------------



In [0]:
target = 'llvm'

input_name = '1'
shape_dict = {input_name: x.shape}
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

with relay.build_config(opt_level=1):
    intrp = relay.build_module.create_executor('graph', mod, tvm.cpu(0), target)

Execute on TVM
---------------------------------------------



In [0]:
dtype = 'float32'
tvm_output = intrp.evaluate()(tvm.nd.array(x.astype(dtype)), **params).asnumpy()

Display results
---------------------------------------------
We put input and output image neck to neck



In [0]:
from matplotlib import pyplot as plt
out_y = Image.fromarray(np.uint8((tvm_output[0, 0]).clip(0, 255)), mode='L')
out_cb = img_cb.resize(out_y.size, Image.BICUBIC)
out_cr = img_cr.resize(out_y.size, Image.BICUBIC)
result = Image.merge('YCbCr', [out_y, out_cb, out_cr]).convert('RGB')
canvas = np.full((672, 672*2, 3), 255)
canvas[0:224, 0:224, :] = np.asarray(img)
canvas[:, 672:, :] = np.asarray(result)
plt.imshow(canvas.astype(np.uint8))
plt.show()